In [24]:
# coding = utf-8 
import numpy as np
import os
import matplotlib.pyplot as plt


'''加载文件'''
def readFile(filePath):
    file = open(filePath,'rb')
    data = file.read()
    file.close()
    return data

'''得到所有文件名,所在目录名'''
def getFileList(rootPath):
    fileList = []
    for root, dirs, files in os.walk(rootPath,topdown = False):#优先遍历子目录
        if len(files) > 0:
            fileList.append((root,files));
    return fileList

'''把得到的加速度,角速度,标签，写入到文件中'''
def write2file(X,Y,X_path,Y_path):
    np.savetxt(X_path,X,delimiter=',')
    np.savetxt(Y_path,Y,delimiter=',')
    
'''从文件中获取加速度和角速度'''
def getAandG(data):
    data_len = len(data)
    a = [[] for i in range(3)]  # 加速度
    w = [[] for i in range(3)]  # 角速度
    index = 0
    while index < data_len:  
        if data[index] == 0x55:  # 包头
            if index + 7 < data_len:
                temp_a = data[index+3]
                temp_b = data[index+5]
                temp_c = data[index+7]
                if data[index+3] > 127:  # 说明是负数
                    temp_a = temp_a - 256
                if data[index+5] >127:
                    temp_b = temp_b - 256
                if data[index+7] >127:
                    temp_c = temp_c - 256
                if data[index+1] == 81: # 加速度输出
                    a[0].append(temp_a*256+data[index+2]) # x轴加速度
                    a[1].append(temp_b*256+data[index+4]) # y轴加速度
                    a[2].append(temp_c*256+data[index+6]) # z轴加速度
                    index += 11
                    continue
                elif data[index+1] == 82: # 角速度输出
                    w[0].append(temp_a*256+data[index+2]) # x轴角速度
                    w[1].append(temp_b*256+data[index+4]) # y轴角速度
                    w[2].append(temp_c*256+data[index+6]) # z轴角速度
                    index += 11
                    continue
                elif data[index+1] == 83: # 角度输出
                    index += 11
                else:  # 或者数据有丢失
                    index += 1
            else:  # 没有完整的数据了
                break
        else:  # 数据有丢失
            index += 1
    a = np.array(a)
    w = np.array(w)
    a = a / 32768 * 16    #单位为g
    w = w / 32768 * 2000  # 单位为°/s
    #数据对齐
    len_diff = len(a[0])-len(w[0])
    if len_diff !=0:
        if len_diff>0:#加速度数据长
            w = np.hstack((w,np.zeros((3,len_diff))))
        elif len_diff<0:#角速度数据长
            a = np.hstack((a,np.zeros((3,-len_diff))))   
    if len(a[0])!=len(w[0]):
        print("数据未对齐")#判断是否对齐
    
    #print(a.shape+w.shape)
    return a,w

'''
读取指定目录原始数据，将其整合成为numpy，
'''
def saveDataToNP(rootPath ,savePath = "DataSet.npy"):
    DataSet = []
    Labels = {"加速":1,"静止":2,"碰撞":3,"匀速":4,"左转":5,"右转":6}
    fileList = getFileList(rootPath)
    print("正在生成文件，请稍后...")  
    for root,files in fileList:
        print(root.split('\\')[-1])
        label = Labels[root.split('\\')[-1]]
        for filePath in files:
            filePath = root+ '/' + filePath
            fileData = readFile(filePath)
            if(len(fileData) == 0):
                continue
            A,G = getAandG(fileData)
            DataSet.append({"Acc":A,"Gyr":G,"Label":label})
    np.save(savePath,np.array(DataSet))
    print("生成文件完毕！")  
'''
生成图片
'''
def generatePic(DataSet,picSavePath = "Pic"):
    Labels = ["加速","静止","碰撞","匀速","左转","右转"]
    axisLabel = ["X","Y","Z"]
    colorLabel = ["r",'g',"b"]
    curLabel = -1
    count = 1
    plt.rcParams['font.sans-serif']=['SimHei'] #用来正常显示中文标签
    plt.rcParams['axes.unicode_minus']=False #用来正常显示负号
    for data in DataSet:
        if curLabel != Labels[data["Label"]-1]:
            count = 1
            curLabel = Labels[data["Label"]-1]#得到中文标签
        else:
            count += 1
        plt.figure(curLabel+"%d"%(count),figsize=(18,12))
        plt.suptitle(curLabel,fontsize=40,x=0.5,y=0.97)
        for i in range(3):
            plt.subplot(3,2,i*2+1)
            plt.title("加速度-"+axisLabel[i])
            plt.plot(range(len(data["Acc"][i])),data["Acc"][i],color=colorLabel[i])
            plt.subplot(3,2,i*2+2)
            plt.title("角速度-"+axisLabel[i])
            plt.plot(range(len(data["Gyr"][i])),data["Gyr"][i],color=colorLabel[i])
        plt.gcf().savefig(picSavePath+"\\"+curLabel+"_%d.png"%(count),dpi = 200,bbox_inches='tight')
    print("生成图片完毕！")    

'''未完成''' 
def LoadDataFromFile(filePath = "DataSet.npy"):
    originData = np.load(filePath)
    

In [25]:
saveDataToNP("DataSet/trim")

正在生成文件，请稍后...
加速
匀速
右转
左转
碰撞
静止
生成文件完毕！


In [3]:
load = np.load("DataSet.npy")

In [29]:
generatePic(load)

C:\Anaconda2\envs\tensorflow\lib\site-packages\matplotlib\pyplot.py:524: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  max_open_warning, RuntimeWarning)


生成图片完毕！
